#Importing Dependencies

In [ ]:
import tensorflow as tf
import pandas as pd
import yfinance as yf
import numpy as np
import math
from itertools import product, combinations
from sklearn.preprocessing import MinMaxScaler
from copy import copy, deepcopy
from collections import deque
from tqdm.auto import tqdm
import os
import pickle as pkl
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import logging
import datetime
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.constraints import unit_norm
from tensorflow.keras.layers import LayerNormalization

Define a Portfolio Environment

In [ ]:
# Define the PortfolioEnv Class
class PortfolioEnv():
    def __init__(self, dates, datasets, n):
        self.dates = dates
        self.datasets = datasets
        self.n = n
        self.process_big_pt()
        self.mm_scaler = MinMaxScaler()
        self.process_small_pt()

        self.initial_pt = 1000000  # 1 million
        self.c_minus = 0.0025  # 0.25% selling transaction cost
        self.c_plus = 0.0025   # 0.25% buying transaction cost
        self.delta = 10000     # 10,000 trading size
        self.current_epsilon = 1
        
        self.process_actions()
        self.action_shape = self.actions.shape[0]
        self._episode_ended = False
        
    def reset(self):
        # Initialization of portfolio
        self.pt = self.initial_pt  # Start with 1 million
        self.wt = np.array([0.25, 0.25, 0.25, 0.25], dtype=np.float64)  # Equal weight portfolio
        self.wt = self.wt / np.sum(self.wt) 

        self.current_tick = 0
        self.episode_ended = False

        # Use index 0 for the first indicator (closing price) - FIXED
        ktc = self.big_pt[self.current_tick, 0, :, 0]
        wt_prime = (self.wt*self.phi(ktc)) / (np.dot(self.wt, self.phi(ktc)))

        return {'big_xt': np.array(self.small_pt[self.current_tick]), 'wt_prime': wt_prime}

    def process_actions(self):
        asset_number = len(self.datasets)
        action_number = len(self.datasets)

        seq = np.arange(asset_number)
        actions = []

        for c in product(seq, repeat=action_number):
            actions.append(c)

        self.actions = np.array(actions)

    def find_action_index(self, action):
        for ind, a in enumerate(self.actions):
            if np.array_equal(a, action):
                return ind
        return None

    def process_big_pt(self):
        """Create proper tensor structure as described in Figure 1"""
        datasets = self.datasets
        date_start = self.dates[0]
        date_end = self.dates[1]

        dfs = []
        for d in datasets:
            ticker = yf.Ticker(d)
            # Get historical market data
            df_ = ticker.history(start=date_start, end=date_end, interval="1d")
            df_.rename(mapper={
                "Close": d+"_close",
                "Open": d+"_open",
                "High": d+"_high",
                "Low": d+"_low",
                "Volume": d+"_volume"
            }, inplace=True, axis=1)
            if "Dividends" in df_.columns:
                df_.drop(axis=1, labels=["Dividends", "Stock Splits"], inplace=True)
            dfs.append(df_)

        final_df = pd.concat(dfs, axis=1)
        final_df.dropna(inplace=True)

        self.final_df = final_df

        n = self.n
        I = len(self.datasets)
    
        # Create the technical indicators per asset
        indicators = []
        for indicator_type in ['c', 'o', 'h', 'l', 'v']:  # 5 indicators
            asset_indicators = []
            for d in self.datasets:  # For each asset
                if indicator_type == 'c':  # Close price change
                    asset_close = self.final_df[d+"_close"].values
                    asset_prev_close = self.final_df[d+"_close"].shift().values
                    K = (asset_close - asset_prev_close) / asset_prev_close
                elif indicator_type == 'o':  # Open price ratio
                    asset_prev_close = self.final_df[d+"_close"].shift().values
                    asset_open = self.final_df[d+"_open"].values
                    K = (asset_open - asset_prev_close) / asset_prev_close
                elif indicator_type == 'h':  # High price ratio
                    asset_close = self.final_df[d+"_close"].values
                    asset_high = self.final_df[d+"_high"].values
                    K = (asset_close - asset_high) / asset_high
                elif indicator_type == 'l':  # Low price ratio  
                    asset_close = self.final_df[d+"_close"].values
                    asset_low = self.final_df[d+"_low"].values
                    K = (asset_close - asset_low) / asset_low
                elif indicator_type == 'v':  # Volume change
                    asset_prev_volume = self.final_df[d+"_volume"].shift().values
                    asset_volume = self.final_df[d+"_volume"].values
                    K = (asset_volume - asset_prev_volume) / asset_prev_volume

                K = K[1:]  # Remove first NaN value
                # Create time windows of size n
                K_windows = [K[i:i+n] for i in range(len(K)-n+1)]
                asset_indicators.append(K_windows)
            indicators.append(asset_indicators)

        # Convert to numpy array and reshape according to Figure 1
        indicators = np.array(indicators)
        # Reshape to [time_steps, window_size(n), assets(I), indicators(5)]
        self.big_pt = indicators.transpose(2, 3, 1, 0)
        print(f"Technical indicators tensor shape: {self.big_pt.shape}")

    def process_small_pt(self):
        """Process technical indicators for small state representation"""
        small_pt = []

        # Step 1: Aggregate all data for fitting
        flattened_all = []
        for big_xt in self.big_pt:
            big_xt = big_xt.swapaxes(0, 1).swapaxes(1, 2)
            flattened_all.append(big_xt.reshape(-1, big_xt.shape[-1]))  # (20, 5)

        all_data = np.vstack(flattened_all)  # shape: (num_samples * 20, 5)
        self.mm_scaler.fit(all_data)

        # Step 2: Transform each sample using the fitted scaler
        for big_xt in self.big_pt:
            big_xt = big_xt.swapaxes(0, 1).swapaxes(1, 2)  # shape: (20, 5)
            big_xt_reshaped = big_xt.reshape(-1, big_xt.shape[-1])
            big_xt_scaled = self.mm_scaler.transform(big_xt_reshaped).reshape(big_xt.shape)
            small_pt.append(big_xt_scaled)

        self.small_pt = np.array(small_pt)
        print(f"Scaled indicators tensor shape: {self.small_pt.shape}")

    def get_30day_volatility(self):
        """Calculate market volatility for adaptive exploration"""
        if self.current_tick < 30:
            return 0.05  # Default volatility when not enough history
        
        # Use close price changes of first asset as market indicator
        window = self.big_pt[self.current_tick-30:self.current_tick, 0, 0, 0]  # Last 30 days
        
        # Calculate daily returns
        returns = np.diff(window) / (window[:-1] + 1e-10)
        
        # Calculate volatility (standard deviation of returns)
        volatility = np.std(returns)
        
        return volatility

    def dynamic_epsilon(self, base=0.5, volatility_scale=1, min_decay=0.995):
        market_vol = self.get_30day_volatility()
        epsilon = base + (1 - base) * np.tanh(volatility_scale * market_vol)
        # Ensure epsilon decays regardless of volatility
        self.current_epsilon = min(self.current_epsilon * min_decay, epsilon) 
        return self.current_epsilon

    def phi(self, v):
        """Operator to increase vector dimension"""
        return np.concatenate(([1], v + 1))

    def get_wt_prime_chapeau(self, wt_prime_no_cash, big_s_minus, big_s_plus, pt_prime):
        """Calculate new weights after action with clearer indexing"""
        wt_prime_chapeau = []
        for ind, key in enumerate(wt_prime_no_cash):
            # Adjust indices to account for the missing cash position
            adj_ind = ind  # The actual asset index in action space
            if adj_ind in big_s_minus:
                wt_prime_chapeau.append(key - self.delta/pt_prime)
            elif adj_ind in big_s_plus:
                wt_prime_chapeau.append(key + self.delta/pt_prime)
            else:
                wt_prime_chapeau.append(key)

        return np.array(wt_prime_chapeau)


    def is_asset_shortage(self, action, pt, wt):
        """Check if there's an asset shortage for selling"""
        action = np.atleast_1d(action)
        big_s_minus = np.where(action == 0)[0]
        for ind in big_s_minus:
            if wt[ind+1] * pt < self.delta:
                return True
        return False

    def is_cash_shortage(self, action, pt, wt):
        """Check if there's a cash shortage for buying"""
        action = np.atleast_1d(action)
        big_s_minus = np.where(action == 0)[0]
        big_s_plus = np.where(action == 2)[0]
        current_cash = wt[0] * pt
        cash_after_selling = current_cash + (1 - self.c_minus) * self.delta * len(big_s_minus)
        cash_needed = (self.c_plus + 1) * self.delta * len(big_s_plus)

        if cash_after_selling < cash_needed:
            return True
        return False

    def action_mapping(self, action, action_Q_values, pt, wt):
        """Implementation of Algorithm 1 from the paper"""
        action = copy(action)
    
        # Check for asset shortage first (as per Algorithm 1)
        if self.is_asset_shortage(action, pt, wt):
            action_mapped = self.rule2(action, action_Q_values, pt, wt)
        # Then check for cash shortage
        elif self.is_cash_shortage(action, pt, wt):
            action_mapped = self.rule1(action, action_Q_values, pt, wt)
        else:
            action_mapped = action
        
        return action_mapped

    def rule1(self, action, action_Q_values, pt, wt):
        """Rule 1: Handle cash shortage case"""
        MAXQ = -np.inf
        action_selected = action
    
        # Identify assets to buy in this action
        big_s_plus = np.where(action == 2)[0]
    
        # Generate power set of buying assets (all possible subsets)
        power_set = []
        for i in range(len(big_s_plus) + 1):
            for c in combinations(big_s_plus, i):
                power_set.append(c)
    
        # Test each subset
        for subset in power_set:
            new_action = copy(action)
            # Convert buy actions to hold for assets in this subset
            for j in subset:
                new_action[j] = 1  # Change to hold
        
            # Check if this new action is feasible
            if not self.is_cash_shortage(new_action, pt, wt):
                # Get Q-value for this action
                new_action_index = self.find_action_index(new_action)
                if new_action_index is not None:
                    new_action_Q_value = action_Q_values[new_action_index]
                
                    # Update if this is the best action so far
                    if new_action_Q_value > MAXQ:
                        MAXQ = new_action_Q_value
                        action_selected = new_action
    
        return action_selected

    def rule2(self, action, action_Q_values, pt, wt):
        """Rule 2: Handle asset shortage case"""
        # Convert sell actions to hold for assets with shortage
        for i in range(len(action)):
            if action[i] == 0:  # Sell action
                if wt[i+1] * pt < self.delta:  # Check if asset shortage
                    action[i] = 1  # Change to hold
    
        # After fixing asset shortage, check if there's now a cash shortage
        if self.is_cash_shortage(action, pt, wt):
            action = self.rule1(action, action_Q_values, pt, wt)
    
        return action

    def simulate_all_feasible_actions(self, state):
        """Simulate all feasible actions in parallel as described in Section 4.2"""
        experiences = []
    
        # Skip simulation if we're at or near the end of our data
        # This prevents trying to access indices beyond our data boundaries
        if self.current_tick >= len(self.big_pt) - 2:
            return experiences
    
        # Get all feasible actions for current state
        feasible_actions = self.F(state['pt'], state['wt_prime'])
    
        # Simulate each feasible action
        for action_idx in feasible_actions:
            action = self.actions[action_idx]
            next_state, reward, done, pt_next, wt_next = self.step(action, simulation=True)
            next_state['pt'] = pt_next
            experiences.append((state, action, reward, next_state, done))
    
        return experiences

    def track_portfolio_composition(self):
        """Return current portfolio composition for debugging"""
        composition = {
            'day': self.current_tick,
            'cash_pct': self.wt[0] * 100,
            'asset_weights': self.wt[1:] * 100,
            'portfolio_value': self.pt
        }
        return composition

    def F(self, pt, wt):
        """Get all feasible actions in current state"""
        action_possible = []
        for ind, action in enumerate(self.actions):
            if not self.is_asset_shortage(action, pt, wt) and not self.is_cash_shortage(action, pt, wt):
                action_possible.append(ind)
        return np.array(action_possible)

    def step(self, action, simulation=False):
        # Check if episode is ending
        if self.current_tick == len(self.big_pt) - 2:
            self.episode_ended = True

        # Step 1: Calculate portfolio value and weights after state evolution but before action
        ktc = self.big_pt[self.current_tick, 0, :, 0]  # Fixed index to 0
        pt_prime = self.pt * np.dot(self.wt, self.phi(ktc))
        wt_prime = (self.wt * self.phi(ktc)) / (np.dot(self.wt, self.phi(ktc)))

        # Step 2: Apply the action
        action = np.atleast_1d(action)
        big_s_minus = np.where(action == 0)[0]
        big_s_plus = np.where(action == 2)[0]

        # Step 3: Calculate transaction costs
        ct = (self.delta * (self.c_minus * len(big_s_minus) + self.c_plus * len(big_s_plus))) / pt_prime
        
        if not simulation:
            pt_after_costs = pt_prime * (1 - ct)
        else:
            pt_after_costs = pt_prime * (1 - ct)

        # Step 4: Calculate new weights after action
        wt_prime_chapeau = self.get_wt_prime_chapeau(wt_prime[1:], big_s_minus, big_s_plus, pt_prime)
        wt_prime_chapeau_0 = wt_prime[0] + self.delta * ((1 - self.c_minus) * len(big_s_minus) - 
                                                         (1 + self.c_plus) * len(big_s_plus)) / pt_prime
        wt_prime_chapeau = np.concatenate((np.array([wt_prime_chapeau_0]), wt_prime_chapeau))

        # Step 5: Apply weight normalization and prepare for next state
        if not simulation:
            # Clip any negative weights to 0
            if all(action == 1):  # Hold action
                self.wt = wt_prime_chapeau  # Let weights drift naturally
            else:
                self.wt = wt_prime_chapeau / np.sum(wt_prime_chapeau)
            # Renormalize so that the sum equals 1
            self.wt = (self.wt * self.phi(ktc)) / np.dot(self.wt, self.phi(ktc))
            self.current_tick += 1
            k_t_plus_one_c = self.big_pt[self.current_tick, 0, :, 0]  # Fixed index to 0
        else:
            if all(action == 1):  # Hold action
                self.wt = wt_prime_chapeau  # Let weights drift naturally
            else:
                self.wt = wt_prime_chapeau / np.sum(wt_prime_chapeau)
            current_tick = self.current_tick + 1
            k_t_plus_one_c = self.big_pt[current_tick, 0, :, 0]  # Fixed index to 0

        # Step 6: Calculate static portfolio value (if no action was taken)
        P_t_plus_one_s = pt_prime * np.dot(wt_prime, self.phi(k_t_plus_one_c))
        
        # Step 7: Calculate actual portfolio value and reward
        if not simulation:
            p_t_plus_one_prime = pt_after_costs * np.dot(self.wt, self.phi(k_t_plus_one_c))
            # Correct reward calculation per the paper
            reward = (p_t_plus_one_prime - P_t_plus_one_s) / P_t_plus_one_s
            #self.pt = p_t_plus_one_prime
            wt_plus_one_prime = (self.wt * self.phi(k_t_plus_one_c)) / (np.dot(self.wt, self.phi(k_t_plus_one_c)))
        else:
            p_t_plus_one_prime = pt_after_costs * np.dot(self.wt, self.phi(k_t_plus_one_c))
            # Correct reward for simulation as well
            reward = (p_t_plus_one_prime - P_t_plus_one_s) / P_t_plus_one_s
            #pt = p_t_plus_one_prime
            wt_plus_one_prime = (self.wt * self.phi(k_t_plus_one_c)) / (np.dot(self.wt, self.phi(k_t_plus_one_c)))
        
        # Step 8: Return next state, reward, and episode status
        if not simulation:
            self.pt = p_t_plus_one_prime
            return {'big_xt': np.array(self.small_pt[self.current_tick]), 
                    'wt_prime': wt_plus_one_prime}, reward, self.episode_ended
        else:
            return {'big_xt': np.array(self.small_pt[current_tick]), 
                    'wt_prime': wt_plus_one_prime}, reward, self.episode_ended, p_t_plus_one_prime, wt_plus_one_prime

Implement Priorotised Buffer Replay Helper classes

In [ ]:

class SumTree:
    """Efficient priority storage structure per Park et al. (2020) Appendix B.3"""
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree_depth = int(np.ceil(np.log2(capacity))) + 1
        self.tree = np.zeros(2 * capacity - 1)  # SumTree array storage
        self.data = np.zeros(capacity, dtype=object)  # Experience storage
        self.ptr = 0  # Circular pointer
        self.size = 0
        
    def _propagate(self, idx, delta):
        parent = (idx - 1) // 2
        # Fix: Ensure delta is a scalar value
        if hasattr(delta, 'item') and np.ndim(delta) > 0:
            delta = delta.item()
        self.tree[parent] += delta
        if parent != 0:
            self._propagate(parent, delta)
            
    def _retrieve(self, idx, value):
        left = 2 * idx + 1
        if left >= len(self.tree):
            return idx
        if value <= self.tree[left]:
            return self._retrieve(left, value)
        else:
            return self._retrieve(left + 1, value - self.tree[left])
        
    def total(self):
        return self.tree[0]
    
    def add(self, priority, data):
        idx = self.ptr + self.capacity - 1
        self.data[self.ptr] = data
        self.update(idx, priority)
        self.ptr = (self.ptr + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)
        
    def update(self, idx, priority):
        # Fix: Ensure priority is a scalar value
        if hasattr(priority, 'item') and np.ndim(priority) > 0:
            priority = priority.item()
        delta = priority - self.tree[idx]
        self.tree[idx] = priority
        self._propagate(idx, delta)
        
    def get(self, value):
        idx = self._retrieve(0, value)
        data_idx = idx - self.capacity + 1
        return (idx, self.tree[idx], self.data[data_idx])

class PrioritizedReplayBuffer:
    """Improved implementation aligned with Park et al. (2020) Section 4.2"""
    def __init__(self, capacity=2000, alpha=0.6, beta=0.4, epsilon=1e-10):
        self.alpha = alpha
        self.beta = beta
        self.epsilon = epsilon
        self.max_priority = 1.0
        self.tree = SumTree(capacity)
        
    def _calculate_priority(self, td_error):
        return (np.abs(td_error) + self.epsilon) ** self.alpha

    def add(self, experience, td_error):
        priority = self._calculate_priority(td_error)
        self.tree.add(priority, experience)
        self.max_priority = max(priority, self.max_priority)

    def sample(self, batch_size, beta_increment=None):
        if self.tree.size == 0:
            return [], [], []

        segment = self.tree.total() / batch_size
        samples = []
        indices = []
        priorities = []
        
        for i in range(batch_size):
            a = segment * i
            b = segment * (i + 1)
            value = np.random.uniform(a, b)
            idx, priority, data = self.tree.get(value)
            samples.append(data)
            indices.append(idx)
            priorities.append(priority)

        # Importance sampling weights
        sampling_probs = np.array(priorities) / self.tree.total()
        is_weights = np.power(self.tree.size * sampling_probs, -self.beta)
        is_weights /= np.max(is_weights)  # Normalize

        # Anneal beta
        if beta_increment:
            self.beta = min(1.0, self.beta + beta_increment)

        return samples, indices, is_weights

    def update_priorities(self, indices, td_errors):
        """Update priorities using TD-errors from latest training batch"""
        priorities = self._calculate_priority(td_errors)
        for idx, priority in zip(indices, priorities):
            self.tree.update(idx, priority)
        self.max_priority = max(priorities.max(), self.max_priority)


You Surely, have to track your metrics!

In [ ]:
class MetricsTracker:
    """Class to track and calculate statistics for various training metrics"""
    def __init__(self):
        self.metrics = defaultdict(list)
        self.episode_metrics = defaultdict(list)
        
    def add(self, metric_name, value):
        """Add a value to the specified metric"""
        self.episode_metrics[metric_name].append(value)
        
    def log_episode(self, episode, year):
        """Calculate statistics for episode metrics and log them"""
        # Calculate statistics for episode metrics
        for metric, values in self.episode_metrics.items():
            if len(values) > 0:
                # Store episode statistics in overall metrics
                self.metrics[f"{metric}_mean"].append(np.mean(values))
                if len(values) > 1:
                    self.metrics[f"{metric}_std"].append(np.std(values))
                else:
                    self.metrics[f"{metric}_std"].append(0)
        
        # Store episode number and year
        self.metrics["episode"].append(episode)
        self.metrics["year"].append(year)
        
        # Reset episode metrics for next episode
        self.episode_metrics = defaultdict(list)
    
    def get_dataframe(self):
        """Convert metrics to pandas DataFrame"""
        return pd.DataFrame(self.metrics)
    
    def plot_metrics(self, save_path="training_metrics.png"):
        """Plot key metrics and save to file"""
        metrics_df = self.get_dataframe()
        
        fig, axes = plt.subplots(4, 2, figsize=(15, 20))
        fig.suptitle('DQN Portfolio Training Metrics', fontsize=16)
        
        # Plot metrics
        if 'cumulative_return' in metrics_df:
            axes[0, 0].plot(metrics_df['episode'], metrics_df['cumulative_return'])
            axes[0, 0].set_title('Cumulative Return (%)')
            axes[0, 0].set_xlabel('Episode')
            axes[0, 0].set_ylabel('Return (%)')
        
        if 'q_values_mean' in metrics_df:
            axes[0, 1].plot(metrics_df['episode'], metrics_df['q_values_mean'])
            axes[0, 1].set_title('Average Q-values')
            axes[0, 1].set_xlabel('Episode')
            axes[0, 1].set_ylabel('Q-value')
            
        if 'reward_mean' in metrics_df:
            axes[1, 0].plot(metrics_df['episode'], metrics_df['reward_mean'])
            axes[1, 0].set_title('Average Reward')
            axes[1, 0].set_xlabel('Episode')
            axes[1, 0].set_ylabel('Reward')
            
        if 'epsilon' in metrics_df:
            axes[1, 1].plot(metrics_df['episode'], metrics_df['epsilon'])
            axes[1, 1].set_title('Epsilon (Exploration Rate)')
            axes[1, 1].set_xlabel('Episode')
            axes[1, 1].set_ylabel('Epsilon')
            
        if 'position_change_mean' in metrics_df:
            axes[2, 0].plot(metrics_df['episode'], metrics_df['position_change_mean'])
            axes[2, 0].set_title('Average Position Change')
            axes[2, 0].set_xlabel('Episode')
            axes[2, 0].set_ylabel('Change')
            
        if 'loss_mean' in metrics_df and 'loss_std' in metrics_df:
            axes[2, 1].errorbar(metrics_df['episode'], metrics_df['loss_mean'], 
                              yerr=metrics_df['loss_std'], alpha=0.7)
            axes[2, 1].set_title('Loss (Mean ± Std)')
            axes[2, 1].set_xlabel('Episode')
            axes[2, 1].set_ylabel('Loss')
            
        if 'action_buy' in metrics_df and 'action_sell' in metrics_df and 'action_hold' in metrics_df:
            axes[3, 0].plot(metrics_df['episode'], metrics_df['action_buy'], label='Buy')
            axes[3, 0].plot(metrics_df['episode'], metrics_df['action_sell'], label='Sell')
            axes[3, 0].plot(metrics_df['episode'], metrics_df['action_hold'], label='Hold')
            axes[3, 0].set_title('Action Counts')
            axes[3, 0].set_xlabel('Episode')
            axes[3, 0].set_ylabel('Count')
            axes[3, 0].legend()
            
        if 'transaction_cost_mean' in metrics_df:
            axes[3, 1].plot(metrics_df['episode'], metrics_df['transaction_cost_mean'])
            axes[3, 1].set_title('Average Transaction Costs')
            axes[3, 1].set_xlabel('Episode')
            axes[3, 1].set_ylabel('Cost')
            
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig(save_path)
        plt.close()
        
        print(f"Training metrics plotted and saved to {save_path}")


Define the Model and Trainning Step Function

In [ ]:
@tf.keras.utils.register_keras_serializable()
class TransposeLayer(tf.keras.layers.Layer):
    def __init__(self, perm=[0, 2, 1], **kwargs):
        super(TransposeLayer, self).__init__(**kwargs)
        self.perm = perm
        
    def call(self, inputs):
        return tf.transpose(inputs, perm=self.perm)
    
    def get_config(self):
        config = super(TransposeLayer, self).get_config()
        config.update({"perm": self.perm})
        return config

# Then modify your build_model function to use the custom layer
def build_model(datasets, action_shape):
    # Modified encoder with custom transpose layer
    shared_encoder = tf.keras.models.Sequential([
        TransposeLayer(name="transpose_layer"),  # Custom layer instead of Lambda
        tf.keras.layers.LSTM(units=128, return_sequences=True, name="shared_lstm1"),
        tf.keras.layers.LSTM(units=20, name="shared_lstm2")
    ])

    # Portfolio weights input
    wt_prime_input = tf.keras.layers.Input(shape=[4], name="weight_input")
    
    # Technical indicator inputs for each asset
    asset_inputs = []
    encoded_assets = []
    
    # Apply the SAME encoder to each asset - now accepting (5, 20) format
    for i, asset in enumerate(datasets):
        asset_input = tf.keras.layers.Input(shape=[5, 20], name=f"asset_{i}_input")
        asset_inputs.append(asset_input)
        
        # Use shared encoder with built-in transpose
        encoded = shared_encoder(asset_input)
        encoded_assets.append(encoded)
    
    # Rest of the model remains the same
    encoded_features = tf.keras.layers.Concatenate(axis=1)(encoded_assets)
    combined_features = tf.keras.layers.Concatenate(axis=1)([encoded_features, wt_prime_input])
    
    hidden1 = tf.keras.layers.Dense(
        256, activation="relu",
        kernel_regularizer=l2(0.001),
        activity_regularizer=l1(0.0005),
        name="hidden1"
    )(combined_features)
    
    hidden1 = LayerNormalization()(hidden1)
    hidden1 = tf.keras.layers.Dropout(0.2)(hidden1)
    
    hidden2 = tf.keras.layers.Dense(
        64, activation="tanh", 
        kernel_constraint=unit_norm(),
        name="hidden2"
    )(hidden1)
    
    output = tf.keras.layers.Dense(
        action_shape,
        name="q_values"
    )(hidden2)
    
    # Create model
    model = tf.keras.models.Model(inputs=[wt_prime_input] + asset_inputs, outputs=output)
    return model

@tf.function
def train_step(asset1, asset2, asset3, wts_prime, mask, target_Q_values, weights):
    """Train step function that works with the modified model"""
    # Cast all inputs to float32 to ensure consistent data types
    asset1 = tf.cast(asset1, tf.float32)
    asset2 = tf.cast(asset2, tf.float32)
    asset3 = tf.cast(asset3, tf.float32)
    wts_prime = tf.cast(wts_prime, tf.float32)
    mask = tf.cast(mask, tf.float32)
    target_Q_values = tf.cast(target_Q_values, tf.float32)
    weights = tf.cast(weights, tf.float32)
    
    with tf.GradientTape() as tape:
        # Pass inputs directly to model - no transposition needed
        all_Q_values = model([wts_prime, asset1, asset2, asset3], training=True)
        
        # Calculate masked Q-values and loss
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        td_errors = tf.abs(target_Q_values - Q_values)

        # Apply importance sampling weights
        loss = tf.reduce_mean(LOSS_FN(target_Q_values, Q_values) * weights)

    # Gradient calculation with clipping
    gradients = tape.gradient(loss, model.trainable_variables)
    gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    
    # Handle NaN gradients
    gradients = [tf.where(tf.math.is_nan(g), 0.0, g) for g in gradients] 
    
    # Apply gradients to model
    OPTIMIZER.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, td_errors

def prepare_inputs_from_state(state):
    """Convert state dictionary to properly shaped model inputs"""
    wt_prime = np.array([state['wt_prime']])
    asset_inputs = []
    for i in range(len(datasets)):
        # No transpose - provide data in original format (5,20)
        asset_data = state['big_xt'][i]
        asset_inputs.append(np.array([asset_data]))
    return [wt_prime] + asset_inputs

def evaluate_model(model, test_env, verbose=True):
    """Evaluate model performance on test environment"""
    test_state = test_env.reset()
    test_state['pt'] = test_env.pt
    
    initial_portfolio_value = test_env.pt
    test_rewards = []
    test_portfolio_values = []
    test_actions = {'buy': 0, 'sell': 0, 'hold': 0}
    test_episode_ended = False
    
    test_pbar = tqdm(total=235, desc="Testing") if verbose else None
    
    # Track drawdown
    max_test_portfolio_value = initial_portfolio_value
    max_test_drawdown = 0
    
    while not test_episode_ended:
        # Use model for action selection (no exploration during testing)
        test_state_inputs = prepare_inputs_from_state(test_state)
        test_Q_values = model.predict(test_state_inputs, verbose=0)[0]
        
        # Calculate Q-value statistics
        q_mean = np.mean(test_Q_values)
        q_std = np.std(test_Q_values)
        
        # Get best action, apply mapping if needed
        best_action_idx = np.argmax(test_Q_values)
        best_action = test_env.actions[best_action_idx]
        
        # Check if best action is feasible
        if (not test_env.is_asset_shortage(best_action, test_state['pt'], test_state['wt_prime']) and 
            not test_env.is_cash_shortage(best_action, test_state['pt'], test_state['wt_prime'])):
            test_action = best_action
        else:
            # Map infeasible action to similar feasible one
            test_action = test_env.action_mapping(best_action, test_Q_values, test_state['pt'], test_state['wt_prime'])
        
        # Count action types
        for a in test_action:
            if a == 0:
                test_actions['sell'] += 1
            elif a == 2:
                test_actions['buy'] += 1
            else:
                test_actions['hold'] += 1
        
        # Take step in test environment
        test_next_state, test_reward, test_episode_ended = test_env.step(test_action)
        test_next_state['pt'] = test_env.pt
        
        # Track metrics
        test_rewards.append(test_reward)
        test_portfolio_values.append(test_env.pt)
        
        # Track drawdown
        if test_env.pt > max_test_portfolio_value:
            max_test_portfolio_value = test_env.pt
        else:
            drawdown = (max_test_portfolio_value - test_env.pt) / max_test_portfolio_value
            max_test_drawdown = max(max_test_drawdown, drawdown)
        
        # Update test state
        test_state = test_next_state
        
        if test_pbar:
            test_pbar.update(1)
    
    if test_pbar:
        test_pbar.close()

    # Calculate performance metrics
    daily_returns = [(v - prev_v) / prev_v for prev_v, v in zip(test_portfolio_values[:-1], test_portfolio_values[1:])]

    # Sharpe Ratio calculation
    risk_free_rate = 0.0001  # daily risk-free rate (≈2.5% annually)
    excess_returns = [r - risk_free_rate for r in daily_returns]
    sharpe_ratio = (np.mean(excess_returns) / np.std(excess_returns)) * np.sqrt(252)

    # Sterling Ratio calculation
    downside_returns = [min(r, 0)**2 for r in daily_returns]
    avg_downside = np.sqrt(np.mean(downside_returns))
    sterling_ratio = (np.mean(excess_returns) / avg_downside) * np.sqrt(252) if avg_downside > 0 else 0

    # Final portfolio value
    final_portfolio_value = test_env.pt
    portfolio_return = (final_portfolio_value / initial_portfolio_value - 1) * 100
    
    # Prepare results
    results = {
        'portfolio_return': portfolio_return,
        'final_value': final_portfolio_value,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_test_drawdown * 100,
        'avg_reward': np.mean(test_rewards),
        'std_reward': np.std(test_rewards),
        'actions': test_actions,
    }
    
    if verbose:
        print(f"Test Results:")
        print(f"  Portfolio Return: {portfolio_return:.2f}%")
        print(f"  Final Portfolio Value: ${final_portfolio_value:.2f}")
        print(f"  Sharpe Ratio: {sharpe_ratio:.4f}")
        print(f"  Sterling Ratio: {sterling_ratio:.4f}")
        print(f"  Maximum Drawdown: {max_test_drawdown*100:.2f}%")
        print(f"  Avg/Std Reward: {np.mean(test_rewards):.4f}/{np.std(test_rewards):.4f}")
        print(f"  Actions - Buy: {test_actions['buy']}, Sell: {test_actions['sell']}, Hold: {test_actions['hold']}")
    
    return results


Main Training Loop

In [ ]:
# Main execution code
if __name__ == "__main__":
    # Set seeds for reproducibility
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    np.random.seed(42)
    
    # Define dataset and parameters
    datasets = ["SPY", "IWD", "IWC"]
    n = 20  # Time window size
    
    # Training dates (multiple years for better generalization)
    train_dates = [
        ["2010-01-01", "2010-12-30"],
        ["2011-01-01", "2011-12-30"],
        ["2012-01-01", "2012-12-30"],
        ["2013-01-01", "2013-12-30"],
        ["2014-01-01", "2014-12-30"],
        ["2015-01-01", "2015-12-30"],
    ]
    
    # Test dates
    test_dates = [
        ["2016-01-01", "2016-12-30"],
    ]
    
    # Create environments
    train_envs = [PortfolioEnv(d, datasets, n) for d in train_dates]
    test_envs = [PortfolioEnv(d, datasets, n) for d in test_dates]
    
    # Use same scaler for test environments
    for test_env in test_envs:
        test_env.mm_scaler = train_envs[0].mm_scaler
    
    # Initialize PrioritizedReplayBuffer (rather than deque)
    buffer = PrioritizedReplayBuffer(capacity=2000, alpha=0.6, beta=0.4)
    
    # Training parameters
    BETA = 0.3  # Parameter for truncated geometric distribution sampling
    EPOCHS = 50
    BATCH_SIZE = 32
    DISCOUNT_RATE = 0.9
    
    # Define loss function and optimizer with reduced learning rate
    LR_SCHEDULE = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4, decay_steps=10000, decay_rate=0.99, staircase=True)
    OPTIMIZER = tf.keras.optimizers.Adam(
        learning_rate=LR_SCHEDULE, beta_1=0.9, beta_2=0.999, epsilon=1e-7, clipnorm=5)
    LOSS_FN = tf.keras.losses.Huber(delta=1)
    
    # Build model and target model
    model = build_model(datasets, train_envs[0].action_shape)
    model.summary()
    TARGET_MODEL = tf.keras.models.clone_model(model)
    TARGET_MODEL.set_weights(model.get_weights())
    
    # Generate truncated geometric distribution for episode sampling
    r = np.random.rand(EPOCHS)
    N = len(train_envs)
    gen_trunc = (N - 1 - np.floor(np.log(1 - r * (1 - (1 - BETA) ** N)) / np.log(1 - BETA))).astype(int)
    metrics_tracker = MetricsTracker()
    # Training loop
    epsilon_values = []
    for epoch, ind_env in enumerate(gen_trunc):
        env = train_envs[ind_env]
        state = env.reset()
        state['pt'] = env.pt
        
        action_counts = {'buy': 0, 'sell': 0, 'hold': 0}
        initial_portfolio_value = env.pt
    
        pbar = tqdm(total=235, desc=f"Training Year {ind_env + 2010}, Episode {epoch+1}/{EPOCHS}")
    
        episode_reward = 0
        episode_ended = False
        epoch_epsilon_values = []
        while not episode_ended:
            # Use dynamic epsilon based on market volatility
            EPSILON = max(0.01, env.dynamic_epsilon(base=0.5, volatility_scale=1))
            epoch_epsilon_values.append(EPSILON)
            metrics_tracker.add('epsilon', EPSILON)
            old_weights = env.wt.copy()

            # Select action (epsilon-greedy)
            if np.random.rand() < EPSILON:
                # Random action from feasible set
                feasible_actions = env.F(state['pt'], state['wt_prime'])
                if len(feasible_actions) > 0:
                    action_idx = np.random.choice(feasible_actions)
                    action = env.actions[action_idx]
                else:
                    action = np.ones(len(env.datasets), dtype=int)
            else:
                # Use model for action selection
                state_inputs = prepare_inputs_from_state(state)
                Q_values = model.predict(state_inputs, verbose=0)[0]
                metrics_tracker.add('q_values', np.mean(Q_values))
                # Get best action, apply mapping if needed
                best_action_idx = np.argmax(Q_values)
                best_action = env.actions[best_action_idx]
                
                # Check if best action is feasible
                if (not env.is_asset_shortage(best_action, state['pt'], state['wt_prime']) and 
                    not env.is_cash_shortage(best_action, state['pt'], state['wt_prime'])):
                    action = best_action
                else:
                    # Map infeasible action to similar feasible one
                    action = env.action_mapping(best_action, Q_values, state['pt'], state['wt_prime'])
            for a in action:
                if a == 0:  # Sell
                    action_counts['sell'] += 1
                elif a == 2:  # Buy
                    action_counts['buy'] += 1
                else:  # Hold
                    action_counts['hold'] += 1
            # Take step in environment
            next_state, reward, episode_ended = env.step(action)
            next_state['pt'] = env.pt
            metrics_tracker.add('reward', reward)

            new_weights = env.wt.copy()
            position_change = np.sum(np.abs(new_weights - old_weights))
            metrics_tracker.add('position_change', position_change)

            # Calculate transaction costs
            big_s_minus = np.where(action == 0)[0]
            big_s_plus = np.where(action == 2)[0]
            transaction_cost = (env.delta * (env.c_minus * len(big_s_minus) + env.c_plus * len(big_s_plus))) / state['pt']
            metrics_tracker.add('transaction_cost', transaction_cost)

            # Store experience in buffer (with initial priority 1.0)
            buffer.add((state, action, reward, next_state, episode_ended), 1.0)
            
            # Simulate all feasible actions for more experience
            all_experiences = env.simulate_all_feasible_actions(state)
            for exp in all_experiences:
                state_exp, action_exp, reward_exp, next_state_exp, done_exp = exp
                buffer.add(exp, 1.0)
            
            # Once buffer has enough samples, train the model
            if buffer.tree.size >= BATCH_SIZE:
                # Sample batch with importance sampling weights
                batch, indices, weights = buffer.sample(BATCH_SIZE)
                
                # Unpack batch
                states, actions, rewards, next_states, dones = zip(*batch)
                
                # Prepare batch inputs properly shaped for the model
                batch_wt_primes = np.array([s['wt_prime'] for s in states])
                batch_preprocessed_states = []
                
                for i in range(len(datasets)):
                    asset_data = []
                    for s in states:
                        # No transpose - provide data in original format (5,20)
                        asset_data.append(s['big_xt'][i])
                    batch_preprocessed_states.append(np.array(asset_data))
                
                batch_preprocessed_states = np.array(batch_preprocessed_states).transpose(1, 0, 2, 3)
                
                # Create action mask (one-hot encoding)
                action_mask = np.zeros((len(batch), env.action_shape))
                for i, a in enumerate(actions):
                    action_idx = env.find_action_index(a)
                    if action_idx is not None:
                        action_mask[i, action_idx] = 1
                
                # Get next state inputs (properly shaped)
                next_wt_primes = np.array([s['wt_prime'] for s in next_states])
                next_preprocessed_states = []
                
                # Modified next state preparation without transposition
                for i in range(len(datasets)):
                    next_asset_data = []
                    for s in next_states:
                        # No transpose - provide data in original format (5,20)
                        next_asset_data.append(s['big_xt'][i])
                    next_preprocessed_states.append(np.array(next_asset_data))

                
                next_preprocessed_states = np.array(next_preprocessed_states).transpose(1, 0, 2, 3)
                
                # Format next state inputs for model
                next_state_inputs = [next_wt_primes]
                for i in range(len(datasets)):
                    next_state_inputs.append(next_preprocessed_states[:, i])
                
                # Double DQN: online model selects action, target model estimates value
                next_Q_values_online = model.predict(next_state_inputs, verbose=0)
                best_next_actions = np.argmax(next_Q_values_online, axis=1)
                next_Q_values_target = TARGET_MODEL.predict(next_state_inputs, verbose=0)
                
                # Calculate target Q-values
                target_Q = np.zeros((len(batch), 1))
                for i, (r, done, next_action_idx) in enumerate(zip(rewards, dones, best_next_actions)):
                    if done:
                        target_Q[i] = r
                    else:
                        target_Q[i] = r + DISCOUNT_RATE * next_Q_values_target[i, next_action_idx]
                
                # Get each asset's data - NO transposition needed
                asset1_data = batch_preprocessed_states[:, 0]  # Shape (batch_size, 5, 20)
                asset2_data = batch_preprocessed_states[:, 1]  # Shape (batch_size, 5, 20)
                asset3_data = batch_preprocessed_states[:, 2]  # Shape (batch_size, 5, 20)

                # Call train_step with raw asset data
                loss, td_errors = train_step(
                    tf.convert_to_tensor(asset1_data),
                    tf.convert_to_tensor(asset2_data),
                    tf.convert_to_tensor(asset3_data),
                    tf.convert_to_tensor(batch_wt_primes),
                    tf.convert_to_tensor(action_mask),
                    tf.convert_to_tensor(target_Q),
                    tf.convert_to_tensor(weights)
                )
                metrics_tracker.add('loss', loss.numpy())
                
                # Update priorities with less restrictive error handling
                td_errors = td_errors.numpy()
                td_errors = np.clip(td_errors, -10.0, 10.0)  # More generous range
                td_errors = np.nan_to_num(td_errors, nan=0.01, posinf=10.0, neginf=-10.0)
                buffer.update_priorities(indices, td_errors)
            
            # Update target network periodically
            if (env.current_tick % 2000 == 0):
                TARGET_MODEL.set_weights(model.get_weights())
            
            # Update state for next iteration
            state = next_state
            episode_reward += reward
            pbar.update(1)
        
        pbar.close()
        cumulative_return = (env.pt / initial_portfolio_value - 1) * 100
        # Log end-of-episode metrics
        metrics_tracker.metrics['cumulative_return'].append(cumulative_return)
        metrics_tracker.metrics['action_buy'].append(action_counts['buy'])
        metrics_tracker.metrics['action_sell'].append(action_counts['sell'])
        metrics_tracker.metrics['action_hold'].append(action_counts['hold'])
    
        # Log episode statistics
        metrics_tracker.log_episode(epoch, ind_env + 2010)
        avg_epoch_epsilon = np.mean(epoch_epsilon_values)
        epsilon_values.append(avg_epoch_epsilon)
        # Print summary
        print(f"Episode {epoch+1} average epsilon: {avg_epoch_epsilon:.4f}")
        print(f"Episode {epoch+1} min/max epsilon: {min(epoch_epsilon_values):.4f}/{max(epoch_epsilon_values):.4f}")
        print(f"Episode {epoch+1} reward: {episode_reward:.4f}")
        print(f"Cumulative return: {cumulative_return:.2f}%")
        print(f"Average Q-value: {metrics_tracker.metrics['q_values_mean'][-1]:.4f} (±{metrics_tracker.metrics['q_values_std'][-1]:.4f})")
        print(f"Average reward: {metrics_tracker.metrics['reward_mean'][-1]:.4f} (±{metrics_tracker.metrics['reward_std'][-1]:.4f})")
        print(f"Average position change: {metrics_tracker.metrics['position_change_mean'][-1]:.4f}")
        print(f"Action counts - Buy: {action_counts['buy']}, Sell: {action_counts['sell']}, Hold: {action_counts['hold']}")
        if 'loss_mean' in metrics_tracker.metrics:
            print(f"Average loss: {metrics_tracker.metrics['loss_mean'][-1]:.4f} (±{metrics_tracker.metrics['loss_std'][-1]:.4f})")
        print("-" * 50)
        print(f"Episode {epoch+1} reward: {episode_reward}")

        if (epoch + 1) % 5 == 0:
            # Get the test environment for 2019
            test_env = test_envs[0]  # The first test environment is for 2019
            test_state = test_env.reset()
            test_state['pt'] = test_env.pt
        
            evaluate_model(model, test_env, verbose=True)


    # Save the final model
    model.save("dqn_portfolio_model.h5")


Plotting Portfolio Weights and Portfolio value evolution

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import datetime
def run_and_plot_portfolio_evolution(model, test_env, save_path="portfolio_evolution_2017.png"):
    """Run the trained model on 2016 data and track portfolio evolution"""
    # Initialize test environment
    test_state = test_env.reset()
    test_state['pt'] = test_env.pt
    
    # Track metrics
    initial_portfolio_value = test_env.pt
    portfolio_track = []
    test_episode_ended = False
    
    # Progress bar for visualization
    pbar = tqdm(total=235, desc="Testing on 2016 Data")
    
    # Run model through entire 2019 trading period
    while not test_episode_ended:
        # Track current portfolio composition
        portfolio_track.append({
            'day': test_env.current_tick,
            'cash': test_env.wt[0],
            'spy': test_env.wt[1],
            'iwd': test_env.wt[2],
            'iwc': test_env.wt[3],
            'portfolio_value': test_env.pt
        })
        
        # Get model prediction for best action
        test_state_inputs = prepare_inputs_from_state(test_state)
        test_Q_values = model.predict(test_state_inputs, verbose=0)[0]
        
        # Select optimal action
        best_action_idx = np.argmax(test_Q_values)
        best_action = test_env.actions[best_action_idx]
        
        # Apply action mapping if needed for feasibility
        if (not test_env.is_asset_shortage(best_action, test_state['pt'], test_state['wt_prime']) and 
            not test_env.is_cash_shortage(best_action, test_state['pt'], test_state['wt_prime'])):
            test_action = best_action
        else:
            test_action = test_env.action_mapping(best_action, test_Q_values, test_state['pt'], test_state['wt_prime'])
        
        # Execute step in environment
        test_next_state, test_reward, test_episode_ended = test_env.step(test_action)
        test_next_state['pt'] = test_env.pt
        
        # Update state for next iteration
        test_state = test_next_state
        pbar.update(1)
    
    # Add final portfolio state
    portfolio_track.append({
        'day': test_env.current_tick,
        'cash': test_env.wt[0],
        'spy': test_env.wt[1],
        'iwd': test_env.wt[2],
        'iwc': test_env.wt[3],
        'portfolio_value': test_env.pt
    })
    
    pbar.close()
    
    # Convert tracking data to DataFrame
    portfolio_df = pd.DataFrame(portfolio_track)
    
    # Create visualization of portfolio evolution
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))
    
    # Plot weights
    ax1.plot(portfolio_df['day'], portfolio_df['cash']*100, label='Cash', linewidth=2, color='blue')
    ax1.plot(portfolio_df['day'], portfolio_df['spy']*100, label='Asset 1 (SPY)', linewidth=2, color='green')
    ax1.plot(portfolio_df['day'], portfolio_df['iwd']*100, label='Asset 2 (IWD)', linewidth=2, color='orange')
    ax1.plot(portfolio_df['day'], portfolio_df['iwc']*100, label='Asset 3 (IWC)', linewidth=2, color='red')
    ax1.set_title('Portfolio Weight Evolution (2019)', fontsize=16)
    ax1.set_xlabel('Trading Days', fontsize=14)
    ax1.set_ylabel('Portfolio Weights (%)', fontsize=14)
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Plot portfolio value
    ax2.plot(portfolio_df['day'], portfolio_df['portfolio_value'], linewidth=2, color='green')
    ax2.set_title('Portfolio Value Evolution (2019)', fontsize=16)
    ax2.set_xlabel('Trading Days', fontsize=14)
    ax2.set_ylabel('Value ($)', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    # Create secondary y-axis for returns percentage
    ax2_2 = ax2.twinx()
    initial_value = portfolio_df['portfolio_value'].iloc[0]
    returns_pct = [(v/initial_value - 1)*100 for v in portfolio_df['portfolio_value']]
    ax2_2.plot(portfolio_df['day'], returns_pct, color='darkgreen', linestyle='--', alpha=0.7)
    ax2_2.set_ylabel('Return (%)', fontsize=14, color='darkgreen')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print summary statistics
    final_portfolio_value = test_env.pt
    portfolio_return = (final_portfolio_value / initial_portfolio_value - 1) * 100
    
    print(f"Final portfolio weights:")
    print(f"  Cash: {test_env.wt[0]*100:.2f}%")
    print(f"  SPY: {test_env.wt[1]*100:.2f}%")
    print(f"  IWD: {test_env.wt[2]*100:.2f}%")
    print(f"  IWC: {test_env.wt[3]*100:.2f}%")
    print(f"Portfolio Return: {portfolio_return:.2f}%")
    print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
    
    # Save portfolio data for further analysis
    portfolio_df.to_csv(f"portfolio_evolution_2019_{datetime.datetime.now().strftime('%Y%m%d')}.csv", index=False)
    
    return portfolio_df

def calculate_buy_and_hold(test_env):
    # Extract price data for the assets
    test_env.reset()
    spy_prices = test_env.final_df['SPY_close'].values
    iwd_prices = test_env.final_df['IWD_close'].values
    iwc_prices = test_env.final_df['IWC_close'].values

    # Initial portfolio value and equal allocation
    initial_total = 1e6
    spy_shares = (initial_total / 3) / spy_prices[0]
    iwd_shares = (initial_total / 3) / iwd_prices[0]
    iwc_shares = (initial_total / 3) / iwc_prices[0]

    # Calculate portfolio value over time
    bh_values = []
    for spy_price, iwd_price, iwc_price in zip(spy_prices, iwd_prices, iwc_prices):
        current_value = (spy_shares * spy_price) + (iwd_shares * iwd_price) + (iwc_shares * iwc_price)
        bh_values.append(current_value)

    return bh_values[:len(test_env.final_df)]  # Ensure alignment with DQN data
def plot_portfolio_evolution_with_bh(portfolio_df, bh_values):
    # Ensure dimensions match
    if len(portfolio_df) != len(bh_values):
        bh_values = bh_values[:len(portfolio_df)]

    # Create the plot
    fig, ax = plt.subplots(figsize=(14, 7))

    # Plot DQN Portfolio
    ax.plot(portfolio_df['day'], portfolio_df['portfolio_value'], label='DQN Portfolio', color='green', linewidth=2)

    # Plot Buy and Hold
    ax.plot(portfolio_df['day'], bh_values, label='Buy & Hold', color='blue', linestyle='--', linewidth=2)

    # Add titles and labels
    ax.set_title('Portfolio Value Comparison (DQN vs. Buy & Hold)', fontsize=16)
    ax.set_xlabel('Trading Days', fontsize=14)
    ax.set_ylabel('Portfolio Value ($)', fontsize=14)
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)

    # Show the plot
    plt.tight_layout()
    plt.show()


# Main execution code
if __name__ == "__main__":
    # Assumes the following variables are already defined:
    # - model: the trained DQN model
    # - test_envs: list of test environments where test_envs[0] is 2019 data
    # - prepare_inputs_from_state: function to format state for model input
    
    print("Running trained DQN model on 2017 test data...")
    test_dates = [
    ["2016-01-01", "2016-12-30"]]
    test_envs = [PortfolioEnv(d, datasets, n) for d in test_dates]
    portfolio_data = run_and_plot_portfolio_evolution(model, test_envs[0])
    
    # For additional analysis, you could calculate:
    # 1. Trading frequency analysis
    buy_count = ((portfolio_data['spy'].diff() > 0) | 
                 (portfolio_data['iwd'].diff() > 0) | 
                 (portfolio_data['iwc'].diff() > 0)).sum()
    
    sell_count = ((portfolio_data['spy'].diff() < 0) | 
                  (portfolio_data['iwd'].diff() < 0) | 
                  (portfolio_data['iwc'].diff() < 0)).sum()
    bh_values = calculate_buy_and_hold(test_envs[0])

    # Plot DQN Portfolio vs. Buy & Hold Strategy
    plot_portfolio_evolution_with_bh(portfolio_data, bh_values)
    
    print(f"\nTrading Analysis:")
    print(f"  Buy actions: {buy_count}")
    print(f"  Sell actions: {sell_count}")
    print(f"  Trading activity rate: {(buy_count + sell_count)/len(portfolio_data)*100:.2f}%")


Computing Sterling Ratio and Sharpe Ratio

In [ ]:
# Evaluate the final model on the test environment and print metrics
results = evaluate_model(model, test_envs[0], verbose=False)

print("Final Model Performance on Test Environment:")
print(f"Portfolio Return: {results['portfolio_return']:.2f}%")
print(f"Final Portfolio Value: ${results['final_value']:.2f}")
print(f"Sharpe Ratio (from evaluate_model): {results['sharpe_ratio']:.4f}")

# Re-run simulation on test_env to record portfolio evolution and compute ratios from scratch
state = test_envs[0].reset()
state['pt'] = test_envs[0].pt
portfolio_values = [test_envs[0].pt]

while True:
    # Use the model to determine best action without exploration
    state_inputs = prepare_inputs_from_state(state)
    Q_values = model.predict(state_inputs, verbose=0)[0]
    best_action_idx = np.argmax(Q_values)
    best_action = test_envs[0].actions[best_action_idx]
    
    # If best action is not feasible, map it to a feasible one
    if (not test_envs[0].is_asset_shortage(best_action, state['pt'], state['wt_prime']) and 
        not test_envs[0].is_cash_shortage(best_action, state['pt'], state['wt_prime'])):
        action = best_action
    else:
        action = test_envs[0].action_mapping(best_action, Q_values, state['pt'], state['wt_prime'])
    
    next_state, reward, done = test_envs[0].step(action)
    next_state['pt'] = test_envs[0].pt
    portfolio_values.append(test_envs[0].pt)
    
    state = next_state
    if done:
        break

portfolio_values = np.array(portfolio_values)
daily_returns = np.diff(portfolio_values) / portfolio_values[:-1]
risk_free_rate = 0.0001  # daily risk-free rate
excess_returns = daily_returns - risk_free_rate

# Calculate Sharpe ratio
computed_sharpe = (np.mean(excess_returns) / np.std(excess_returns)) * np.sqrt(252)

# Calculate Sterling ratio
downside_returns = np.array([min(r, 0) ** 2 for r in daily_returns])
avg_downside = np.sqrt(np.mean(downside_returns))
computed_sterling = (np.mean(excess_returns) / avg_downside) * np.sqrt(252) if avg_downside > 0 else 0

print(f"Recomputed Sharpe Ratio: {computed_sharpe:.4f}")
print(f"Sterling Ratio: {computed_sterling:.4f}")

Computing B&H Returns for all 4 years

In [ ]:
# Define the date ranges for each year
years = {
    "2016": ["2016-01-01", "2016-12-30"],
    "2017": ["2017-01-01", "2017-12-30"],
    "2018": ["2018-01-01", "2018-12-30"],
    "2019": ["2019-01-01", "2019-12-30"],
}

# Dictionary to hold results
results = {}

# Loop through each year
for yr, dates in years.items():
    print(f"\nYear {yr}:")
    # Create a PortfolioEnv for the given year (datasets and n are defined in earlier cells)
    env = PortfolioEnv(dates, datasets, n)
    
    # Calculate Buy & Hold portfolio values over the period
    env.reset()
    env.wt = np.array([0.25, 0.25, 0.25, 0.25])  # Start with all cash
    bh_values = calculate_buy_and_hold(env)
    
    # Compute overall return (%)
    initial_value = bh_values[0]
    final_value = bh_values[-1]
    return_pct = (final_value / initial_value - 1) * 100
    
    # Compute daily returns
    bh_values_arr = np.array(bh_values)
    daily_returns = np.diff(bh_values_arr) / bh_values_arr[:-1]
    
    # Set risk free rate (daily)
    risk_free_rate = 0.0001  # daily risk-free rate
    
    # Excess daily returns
    excess_returns = daily_returns - risk_free_rate
    
    # Sharpe Ratio (annualized)
    if np.std(excess_returns) > 0:
        sharpe = (np.mean(excess_returns) / np.std(excess_returns)) * np.sqrt(252)
    else:
        sharpe = 0.0
    
    # Sterling Ratio: calculate downside risk
    downside_returns = np.array([min(r, 0)**2 for r in daily_returns])
    avg_downside = np.sqrt(np.mean(downside_returns))
    if avg_downside > 0:
        sterling = (np.mean(excess_returns) / avg_downside) * np.sqrt(252)
    else:
        sterling = 0.0
    
    # Save results
    results[yr] = {
        "Return (%)": return_pct,
        "Final Value": final_value,
        "Sharpe Ratio": sharpe,
        "Sterling Ratio": sterling
    }
    
    # Print the results
    print(f"  Buy & Hold Return: {return_pct:.2f}%")
    print(f"  Final Portfolio Value: ${final_value:,.2f}")
    print(f"  Sharpe Ratio: {sharpe:.4f}")
    print(f"  Sterling Ratio: {sterling:.4f}")

# Optionally, print the dictionary containing all results.
print("\nSummary:")
print(results)